## Approah 1 

In [54]:
import pandas as pd
import numpy as np

In [55]:
train_articles = pd.read_csv("/data/semeval-2020/task-11/processed/train_article.csv")

In [56]:
spans = pd.read_csv("/data/semeval-2020/task-11/processed/span_labels.csv")

In [57]:
train_articles.head()

,article_id,content
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...
1,787529309,The Last-Minute Character Assassination of Jud...
2,999001296,Altered Election Documents Tied To Florida Dem...
3,729348908,Virginia man who wanted to join ISIS pleads gu...
4,111111134,Paul Manafort Secretly Met With Julian Assange...


In [58]:
spans.head()

,article_id,start,end,label
0,776616374,149,174,Exaggeration_Minimisation
1,776616374,751,786,Exaggeration_Minimisation
2,776616374,831,849,Name_Calling_Labeling
3,776616374,2641,2650,Flag-Waving
4,776616374,2771,2782,Loaded_Language


In [7]:
spans["start_end"] = spans.apply(lambda x:(x.start,x.end,x.label),axis=1)

In [8]:
spans.head()

,article_id,start,end,label,start_end
0,776616374,149,174,Exaggeration_Minimisation,"(149, 174, Exaggeration_Minimisation)"
1,776616374,751,786,Exaggeration_Minimisation,"(751, 786, Exaggeration_Minimisation)"
2,776616374,831,849,Name_Calling_Labeling,"(831, 849, Name_Calling_Labeling)"
3,776616374,2641,2650,Flag-Waving,"(2641, 2650, Flag-Waving)"
4,776616374,2771,2782,Loaded_Language,"(2771, 2782, Loaded_Language)"


In [9]:
article_with_span = train_articles.merge(spans,on="article_id")

In [10]:
article_with_span["article_length"] = article_with_span.content.apply(lambda x: len(x.split(" ")))

In [11]:
article_with_span.head(100)

,article_id,content,start,end,label,start_end,article_length
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...,630,669,Exaggeration_Minimisation,"(630, 669, Exaggeration_Minimisation)",392
1,787529309,The Last-Minute Character Assassination of Jud...,70,88,Loaded_Language,"(70, 88, Loaded_Language)",1568
2,787529309,The Last-Minute Character Assassination of Jud...,288,320,Loaded_Language,"(288, 320, Loaded_Language)",1568
3,787529309,The Last-Minute Character Assassination of Jud...,924,961,Loaded_Language,"(924, 961, Loaded_Language)",1568
4,787529309,The Last-Minute Character Assassination of Jud...,2405,2431,Loaded_Language,"(2405, 2431, Loaded_Language)",1568
...,...,...,...,...,...,...,...
95,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,1969,1979,Loaded_Language,"(1969, 1979, Loaded_Language)",710
96,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,2018,2195,Whataboutism_Straw_Men_Red_Herring,"(2018, 2195, Whataboutism_Straw_Men_Red_Herring)",710
97,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,2566,2577,Loaded_Language,"(2566, 2577, Loaded_Language)",710
98,789121265,Juanita Broaddrick: Senator Feinstein Had No I...,2848,2893,Doubt,"(2848, 2893, Doubt)",710


In [12]:
def create_label_seq(row):
    article_id = row.article_id
    if (article_id == 762956953):
        return []
    article = row.content
    print(article_id)
    len_article = len(article.split(" "))
    
    article_labels_seq = []
    article_span_seq = []
    article_no_propaganda_seq = []
    
    for i in range(len_article):
        article_labels_seq.append("O")
    
    spans_for_article = list(set(spans[spans["article_id"] == article_id].start_end.values.tolist()))
    print(spans_for_article)
    for start_end in spans_for_article:
        start = start_end[0]
        end = start_end[1]
        propaganda = start_end[2]
        for ix in range(start,end):
            
            
            
            new_tag = None
            print(len(article_labels_seq),start,end)
            current_tag = article_labels_seq[ix]
            if current_tag == "O":
                article_labels_seq[ix] = propaganda
            else:
                current_tags = current_tag.split(",")
                current_tags.append(propaganda)
                current_tag_string = ",".join(current_tags)
                article_labels_seq[ix] = current_tag_string
    
    return article_labels_seq

In [ ]:
train_articles["label"] = train_articles.apply(lambda x: create_label_seq(x),axis=1)

In [14]:
len(train_articles[train_articles["article_id"] == 787529309].content[1].split(" "))

1568

In [15]:
spans[spans["article_id"] == 787529309]

,article_id,start,end,label,start_end
1539,787529309,70,88,Loaded_Language,"(70, 88, Loaded_Language)"
1540,787529309,288,320,Loaded_Language,"(288, 320, Loaded_Language)"
1541,787529309,924,961,Loaded_Language,"(924, 961, Loaded_Language)"
1542,787529309,2405,2431,Loaded_Language,"(2405, 2431, Loaded_Language)"
1543,787529309,2694,2708,Loaded_Language,"(2694, 2708, Loaded_Language)"
1544,787529309,3158,3173,Loaded_Language,"(3158, 3173, Loaded_Language)"
1545,787529309,8209,8232,Name_Calling_Labeling,"(8209, 8232, Name_Calling_Labeling)"
1546,787529309,8337,8366,Name_Calling_Labeling,"(8337, 8366, Name_Calling_Labeling)"
1547,787529309,8567,8586,Loaded_Language,"(8567, 8586, Loaded_Language)"
1548,787529309,8686,8711,Loaded_Language,"(8686, 8711, Loaded_Language)"


In [16]:
labels = ["O","O"]
np.array(labels)

array(['O', 'O'], dtype='<U1')

In [17]:
list(range(3,6))

[3, 4, 5]

## Approach 2 

In [191]:
import pandas as pd
import glob
import swifter
path = r'/data/semeval-2020/task-11/datasets/train-tagged_article/'
all_files = glob.glob(path + "/*.txt")

content_tuple = []

for filename in all_files:
    file = open(filename)
    content = "".join(file.readlines()).replace("\n"," ")
    article_id = filename.split("article")[-1].split(".txt")[0]
    content_tuple.append((article_id,content))

In [192]:
content_df = pd.DataFrame(content_tuple,columns=["article_id","content"])

In [193]:
content_df.head(3)

,article_id,content
0,762956953,Iran Admits To Aiding Al-Qaeda and Facilitatin...
1,787529309,The Last-Minute <span-11 Character Assassinati...
2,999001296,Altered Election Documents Tied To Florida Dem...


In [266]:
import re

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English   # updated , 
from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
tokenizer = Tokenizer(nlp.vocab)

def get_technique_id(token):
    results = {"start_token":[],"end_token":[]}

    if "<span" in token:
        results["start_token"] = [i.replace("<span-","") for i in re.findall("<span-\d+",token)]

    if "-/span>" in token:
        results["end_token"] = [i.replace("-/span>","") for i in re.findall("\d+-/span>",token)]

    # return [res for res_array in results_start_span for res in res_array]
    return results

def get_propaganda_sequence(row):
#     print(content)
    content_lable_tup = []
    content = row.content.replace("><", "> <").replace(">“<", "> <") \
        .replace(">(<", "> <").replace(">.<", "> <") \
        .replace(">\'<", "> <")
    #     article_id = row.article_id
    #     print(article_id)
    label_seq = []
    label_propaganda_seq = []
    running_token = []
    for sent in nlp(content).sents:
        len_of_sentences = len(str(sent).split(" "))
        tokens = sent.text.split(" ")
#         tokens = tokenizer.tokenize(str(sent))
        for ix,token in enumerate(tokens):
            token = token.strip()
            modified_token = token
            if "span" in token:
                if "-/span>" in token:
                    #             print(running_token)
                    #             print(get_technique_id(token))
                    for i in get_technique_id(token)["end_token"]:
                        running_token.remove(i)
                    running_token = sorted(running_token)

                    rest_of_token = re.sub("\d+-/span>", "", token.strip())
                    if len(rest_of_token) > 1:
                        split_toks = [i.text for i in nlp(rest_of_token)]
                        for tok in split_toks:
                            if len(running_token) == 0:
                                label_propaganda_seq.append("O")
                                content_lable_tup.append((tok,"O"))
                            else:
                                label_propaganda_seq.append(",".join(running_token))
                                content_lable_tup.append((tok, ",".join(running_token)))

                if "<span" in token:
                    running_token = running_token + get_technique_id(token)["start_token"]
                    running_token = sorted(running_token)

                    rest_of_token = re.sub("<span-\d+", "", token.strip())
                    if len(rest_of_token) > 1:
                        split_toks = [i.text for i in nlp(rest_of_token)]
                        for tok in split_toks:
                            if len(running_token) == 0:
                                label_propaganda_seq.append("O")
                                content_lable_tup.append((tok, "O"))
                            else:
                                label_propaganda_seq.append(",".join(running_token))
                                content_lable_tup.append((tok, ",".join(running_token)))

            else:
                if len(token) > 0:
                    split_toks = [i.text for i in nlp(token)]
                    for tok in split_toks:
                        if len(running_token) == 0:
                            label_propaganda_seq.append("O")
                            content_lable_tup.append((tok, "O"))
                        else:
                            label_propaganda_seq.append(",".join(running_token))
                            content_lable_tup.append((tok, ",".join(running_token)))
            if ix == len_of_sentences -1:
                content_lable_tup[-1] = (content_lable_tup[-1][0] + "$$$$$$",content_lable_tup[-1][1])
                    
#     d = nlp(content)
#     sentences = [sent.string.strip() for sent in d.sents]
#     s = [i + "$$$$$$" for i in sentences]

    # joined_content = " ".join([i[0] for i in content_lable_tup])
    # d = nlp(joined_content)
    # sentences = [sent.string.strip() for sent in d.sents]
    # s = [i + "$$$$$$" for i in sentences]
#     print(content_lable_tup)
#     for t in content_lable_tup:
#         print(t[0])
    new_content = [(re.sub("<span-\d+", "", re.sub("\d+-/span>", "", tup[0])) , tup[1]) for tup in content_lable_tup]

    seq_to_return = [i for i in new_content if i[0] != ""]

    return seq_to_return

In [267]:
# get_propaganda_sequence(content_df[content_df["article_id"] == "736231219"].iloc[0].content)

In [268]:
a = nlp("This is a 'sentence")

In [269]:
print([i.text for i in a])

['This', 'is', 'a', "'", 'sentence']


In [270]:
get_technique_id("<span-11")

{'start_token': ['11'], 'end_token': []}

In [271]:
# get_propaganda_sequence({"content":content_df.iloc[2].content,"article_id":11})

In [272]:
content_df["tagged_sequence_with_propaganda_types"] = content_df.swifter.apply(get_propaganda_sequence,axis=1)

/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [273]:
content_df["tagged_sequence_with_propaganda_types"][1]

[('The', 'O'),
 ('Last', 'O'),
 ('-', 'O'),
 ('Minute', 'O'),
 ('Character', '11'),
 ('Assassination', '11'),
 ('of', 'O'),
 ('Judge', 'O'),
 ('Kavanaugh', 'O'),
 ('Using', 'O'),
 ('any', 'O'),
 ('despicable', '8'),
 ('tactic', '8'),
 ('at', 'O'),
 ('hand', 'O'),
 ('to', 'O'),
 ('derail', 'O'),
 ('Judge', 'O'),
 ('Brett', 'O'),
 ('Kavanaugh', 'O'),
 ('’s', 'O'),
 ('Supreme', 'O'),
 ('Court', 'O'),
 ('confirmation', 'O'),
 ('less', 'O'),
 ('than', 'O'),
 ('a', 'O'),
 ('week', 'O'),
 ('before', 'O'),
 ('the', 'O'),
 ('Senate', 'O'),
 ('Judiciary', 'O'),
 ('Committee', 'O'),
 ('is', 'O'),
 ('scheduled', 'O'),
 ('to', 'O'),
 ('vote', 'O'),
 ('on', 'O'),
 ('whether', 'O'),
 ('to', 'O'),
 ('approve', 'O'),
 ('his', 'O'),
 ('nomination', 'O'),
 (',', 'O'),
 ('Senate', 'O'),
 ('Democrats', 'O'),
 ('have', '8'),
 ('sunk', '8'),
 ('to', '8'),
 ('their', '8'),
 ('lowest', '8'),
 ('level', '8'),
 ('of', 'O'),
 ('character', '11'),
 ('assassination', '11'),
 ('yet', 'O'),
 ('.$$$$$$', 'O'),
 ('They

In [274]:
def propaganda_type_seq_to_simple_yes_or_no_seq(taggings):
    simple_taggings = []
    for i in taggings:
        if i[1] != "O":
            simple_taggings.append((i[0],"I"))
        else:
            simple_taggings.append(i)
    return simple_taggings        

In [275]:
def propaganda_type_seq_to_simple_yes_or_no_seq_bio(taggings):
    simple_taggings = []
    for ix,i in enumerate(taggings):
        if i[1] != "O" and taggings[ix-1][1] == "O" and ix > 1:
            simple_taggings.append((i[0],"B-I"))
        elif i[1] != "O":    
            simple_taggings.append((i[0],"I"))
        else:
            simple_taggings.append(i)
    return simple_taggings        

In [276]:
content_df["tagged_sequence"] = content_df.tagged_sequence_with_propaganda_types.swifter.apply(propaganda_type_seq_to_simple_yes_or_no_seq)

In [277]:
# for i in zip(content_df.iloc[1]["tagged_sequence_with_propaganda_types"],content_df.iloc[1]["tagged_sequence"]):
#     print(i)

In [278]:
from sklearn.model_selection import train_test_split

In [279]:
train_df , test_df =  train_test_split(content_df,test_size=0.2)

In [280]:
last_article = content_df["content"][254]

In [281]:
last_article

'Keith Ellison Defends Louis Farrakhan: "He Had Something To Offer"  Rep. Keith Ellison (D-MN), aka Hakim Muhammad, recently defended his ties to Nation of Islam leader Louis Farrakhan, who recently <span-8 paralleled Jews with termites 8-/span>. During a debate with Republican opponent Doug Wardlow, Ellison was asked about his previous support of Farrakhan, but then claims that he has distanced himself from Farrakhan. <span-8 Yeah, right, Hakim 8-/span>! Understand that Ellison attempts to tell the audience and his opponent that he has distanced himself from Farrakhan since the 1990s. Take a look at his comments. take our poll - story continues below Should military force be used to stop the caravan of migrants marching toward the U.S. border? Should military force be used to stop the caravan of migrants marching toward the U.S. border? Should military force be used to stop the caravan of migrants marching toward the U.S. border? * Yes, military force should be used. No, keep the mili

In [282]:
# get_propaganda_sequence(last_article)

In [283]:
train_tag_array = train_df.tagged_sequence.values.tolist()
test_tag_array = test_df.tagged_sequence.values.tolist()

In [284]:
# final_string = ""
# for i in train_tag_array:
#     for k in i:
#         final_string = final_string + k[0] + " " + k[1] + "\n"
#     final_string = final_string + "\n"    

In [285]:
# with open('train_task.txt','w') as f:
#     f.write(final_string)

In [286]:
# test_final_string = ""
# for i in test_tag_array:
#     for k in i:
#         test_final_string = final_string + k[0] + " " + k[1] + "\n"
#     test_final_string = final_string + "\n"    

In [287]:
# with open('dev_task.txt','w') as f:
#     f.write(test_final_string)

In [288]:
# train_df["article_length"] = train_df.content.swifter.apply(lambda x:len(x.split(" ")))

In [289]:
# max(train_df.article_length.values.tolist())

In [290]:
final_string = ""
count = len(train_tag_array)
for i in train_tag_array:
    final_string += "SOA O\nSOS O\n"
    for k in i:
        final_string = final_string + k[0].replace("$$$$$$","") + " " + k[1] + "\n"
        if "$$$$$$" in k[0]:
            final_string = final_string + "EOS O\n\nSOS O\n"
    final_string += "EOA O\n\n"

In [291]:
with open('train_task.txt','w') as f:
    f.write(final_string)

In [292]:
test_final_string = ""
for i in test_tag_array:
    test_final_string += "SOA O\nSOS O\n"
    for k in i:
        test_final_string = test_final_string + k[0].replace("$$$$$$","") + " " + k[1] + "\n"
        if "$$$$$$" in k[0]:
            test_final_string = test_final_string + "EOS O\n\nSOS O\n"
    test_final_string += "EOA O\n\n"    

In [293]:
with open('dev_task.txt','w') as f:
    f.write(test_final_string)

## Prepare test dataset

In [3]:
dev_articles_path = "/data/semeval-2020/task-11/datasets/dev-articles"

In [4]:
import pandas as pd
import glob
import swifter
all_files = glob.glob(dev_articles_path + "/*.txt")

dev_content_tuple = []

for filename in all_files:
    file = open(filename)
    content = "".join(file.readlines()).replace("\n"," ")
    article_id = filename.split("article")[-1].split(".txt")[0]
    dev_content_tuple.append((article_id,content))

In [5]:
dev_content_tuple[0]

('779309765',
 "Unbelievable! Sharia New Mexico: Islamic compound jihadis RELEASED on bond after charges of “Islamophobia” and “racism” (Islam is not a race)  Editor's Note: Talk about injustice! There were remains of a 4-year-old boy found there, who allegedly died while they were performing some sort of Islamic ritual over him. The Bundys and their supporters got nearly two years in jail and no one hurt a single person! Shame on those who are supposed to uphold law and justice! Shame! Taos County Sheriff Jerry Hogrefe testified that they found children holding boxes of ammunition, and that one child was found with a gun…. Because sharia trumps dead children, school shooting training, kidnapping and jihad training. The children discovered at an “extremist Muslim” compound in New Mexico earlier this month were both trained to use firearms and taught multiple tactical techniques in order to kill teachers, law enforcement and other institution…. state prosecutors said on Monday. take our

In [6]:
len(dev_content_tuple)

75

In [7]:
from spacy.lang.en import English # updated
from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated

article_id_sentence = [(i[0],str(k)) for i in dev_content_tuple for k in nlp(i[1]).sents]

In [8]:
dev_article_text = ""
for i in dev_content_tuple:
    dev_article_text += "SOA\n"
    for sent in nlp(i[1]).sents:
        dev_article_text += "SOS\n"
        tokens = str(sent).split(" ")
#         tokens = tokenizer.tokenize(str(sent))
        for token in tokens:
            for tok in nlp(token):
                dev_article_text = dev_article_text + tok.text + "\n"
        dev_article_text = dev_article_text + "EOS\n\n"
    dev_article_text = dev_article_text + "EOA\n\n"    

In [9]:
# dev_article_text = ""
# for combo in article_id_sentence:
#     dev_article_text += "SOA O\n"
#     for token in combo[1].split(" "):
#         dev_article_text = dev_article_text + token + "\n"
#     dev_article_text = dev_article_text + "\n"    

In [10]:
with open('test_task.txt',"w") as f:
    f.write(dev_article_text)

In [11]:
dev_article_text.split("\n\n")[0]

'SOA\nSOS\nUnbelievable\n!\nEOS'

## Post processing for bert transformers



In [12]:
final_prediction = open("final_prediction.txt").readlines()

In [13]:
article_ids = [tup[0] for tup in dev_content_tuple]

In [14]:
article_prediction = []
running_article = []
for line in final_prediction:
    if line.startswith("SOA") and len(running_article) != 0:
        article_prediction.append(running_article)
        running_article = []
    else: 
        running_article.append(line)
article_prediction.append(running_article)        

In [15]:
dev_content_tuple[-1]

('787966255',
 "Warrants Show Police Never Searched Amber Guyger’s Apartment, Now It’s Too Late, She’s ‘Vacated’ It  Dallas, TX — On Sunday, activists rallied around the Dallas Cowboy’s stadium to draw attention to the slaying of Botham Jean. They carried coffins and blocked traffic to expose the special treatment of Amber Guyger, the cop who has yet to even be fired for killing Jean. Now, as more details emerge, this special treatment has moved to a level that is even more insidious. Protest at AT&T Stadium pic.twitter.com/lu5VsGQByv — Allison Harris (@AllisonFox4News) September 16, 2018 take our poll - story continues below Who should replace Nikki Haley as our ambassador to the U.N.? Who should replace Nikki Haley as our ambassador to the U.N.? Who should replace Nikki Haley as our ambassador to the U.N.? * John Bolton Richard Grenell Dina Powell Heather Nauert Ivanka Trump Email * Email This field is for validation purposes and should be left unchanged. Completing this poll grants 

In [16]:
def create_equi_length_seq(line):
    cleaned_line = line.replace("\n","").split(" ")
    token = cleaned_line[0].strip()
    if len(token) == 0 or token.startswith("SOA") or token.startswith("SOS") or token.startswith("EOA") or token.startswith("EOS"):
        return (" "," ")
    token_label = cleaned_line[1].strip()
    len_token = len(token)
    full_token_label = []
    for i in range(0,len_token):
        full_token_label.append(token_label)
    return (token,"".join(full_token_label))

In [17]:
create_equi_length_seq('trust O\n')

('trust', 'OOOOO')

In [18]:
create_equi_length_seq(' \n')

(' ', ' ')

In [19]:
final_sequence_tups = []
running_article = []
for ap in article_prediction:
    for word_tup in ap:
        resized_tup = create_equi_length_seq(word_tup)
        if (resized_tup[0] != " "):
            running_article.append(resized_tup)
    final_sequence_tups.append(running_article)

In [20]:
final_prediction_seq = []
for article_words_tup in final_sequence_tups:
    label_seq = []
    for word_tup in article_words_tup:
        label_seq.append(word_tup[1])
    final_prediction_seq.append(" ".join(label_seq))
    label_seq = []

In [21]:
article_final_sequence = list(zip(article_ids,final_prediction_seq))

In [22]:
def get_spans_from_label_seq(article_id_label_seq_tup):
    article_id = article_id_label_seq_tup[0]
    label_sequence = article_id_label_seq_tup[1]
    spans = []
    current_running_start_span = None
    is_span_running = False
    for ix,label in enumerate(label_sequence):
        if is_span_running:
            if label == "O":
                spans.append((current_running_start_span,ix-1))
                current_running_start_span = None
                is_span_running = False
        else:
            if label == "I":
                current_running_start_span = ix
                is_span_running = True
    return article_id, spans            

In [23]:
get_spans_from_label_seq(article_final_sequence[0])

('779309765', [(158816, 159110), (188194, 188281), (335245, 335661)])

In [24]:
# create_article_id_label_seqfor article_seq in article_final_sequence
final_submission_list =  [get_spans_from_label_seq(article_tup) for article_tup in article_final_sequence]

In [25]:
final_submission_strings = []
for fsl in final_submission_list:
    article_id = fsl[0]
    for span in fsl[1]:
        final_submission_strings.append(str(article_id) + "\t" + str(span[0]) + "\t" + str(span[1]))

In [26]:
final_submission_content = "\n".join(final_submission_strings)

In [27]:
final_submission_content = "id\tbegin_offset\tend_offset\n" + final_submission_content

In [28]:
with open("final_submission.txt","w") as f:
    f.writelines(final_submission_content)

## Post processing for flair output


In [13]:
final_prediction = open("/home/raghavan/projects/semeval2020/task11/notebooks/resources/taggers/example-ner/test.tsv").readlines()

In [14]:
article_ids = [tup[0] for tup in dev_content_tuple]

In [15]:
article_prediction = []
running_article = []
for line in final_prediction:
    if line.startswith("SOA") and len(running_article) != 0:
        article_prediction.append(running_article)
        running_article = []
    else: 
        running_article.append(line)
article_prediction.append(running_article)        

In [16]:
dev_content_tuple[-1]

('787966255',
 "Warrants Show Police Never Searched Amber Guyger’s Apartment, Now It’s Too Late, She’s ‘Vacated’ It  Dallas, TX — On Sunday, activists rallied around the Dallas Cowboy’s stadium to draw attention to the slaying of Botham Jean. They carried coffins and blocked traffic to expose the special treatment of Amber Guyger, the cop who has yet to even be fired for killing Jean. Now, as more details emerge, this special treatment has moved to a level that is even more insidious. Protest at AT&T Stadium pic.twitter.com/lu5VsGQByv — Allison Harris (@AllisonFox4News) September 16, 2018 take our poll - story continues below Who should replace Nikki Haley as our ambassador to the U.N.? Who should replace Nikki Haley as our ambassador to the U.N.? Who should replace Nikki Haley as our ambassador to the U.N.? * John Bolton Richard Grenell Dina Powell Heather Nauert Ivanka Trump Email * Email This field is for validation purposes and should be left unchanged. Completing this poll grants 

In [30]:
def create_equi_length_seq(line):
    cleaned_line = line.replace("\n","").split(" ")
    print(cleaned_line)
    token = cleaned_line[0].strip()
    if len(token) == 0 or token.startswith("SOA") or token.startswith("SOS") or token.startswith("EOA") or token.startswith("EOS"):
        return (" "," ")
    token_label = cleaned_line[2].strip()
    len_token = len(token)
    full_token_label = []
    for i in range(0,len_token):
        full_token_label.append(token_label)
    return (token,"".join(full_token_label))

In [33]:
create_equi_length_seq('Unbelievable!  O 0.49536633491516113')

('Unbelievable!', 'OOOOOOOOOOOOO')

In [34]:
final_sequence_tups = []
running_article = []
for ap in article_prediction:
    for word_tup in ap:
        resized_tup = create_equi_length_seq(word_tup)
        if (resized_tup[0] != " "):
            running_article.append(resized_tup)
    final_sequence_tups.append(running_article)

In [35]:
final_prediction_seq = []
for article_words_tup in final_sequence_tups:
    label_seq = []
    for word_tup in article_words_tup:
        label_seq.append(word_tup[1])
    final_prediction_seq.append(" ".join(label_seq))
    label_seq = []

In [36]:
article_final_sequence = list(zip(article_ids,final_prediction_seq))

In [37]:
def get_spans_from_label_seq(article_id_label_seq_tup):
    article_id = article_id_label_seq_tup[0]
    label_sequence = article_id_label_seq_tup[1]
    spans = []
    current_running_start_span = None
    is_span_running = False
    for ix,label in enumerate(label_sequence):
        if is_span_running:
            if label == "O":
                spans.append((current_running_start_span,ix-1))
                current_running_start_span = None
                is_span_running = False
        else:
            if label == "I":
                current_running_start_span = ix
                is_span_running = True
    return article_id, spans            

In [38]:
get_spans_from_label_seq(article_final_sequence[0])

('779309765',
 [(615, 708),
  (1500, 1602),
  (2095, 2338),
  (2692, 2831),
  (2878, 2933),
  (2936, 3025),
  (7325, 7435),
  (7582, 7692),
  (7741, 7800),
  (8070, 8167),
  (8392, 8636),
  (8913, 9050),
  (9146, 9226),
  (10223, 10294),
  (10333, 10443),
  (10910, 11274),
  (11422, 11535),
  (11550, 11617),
  (11888, 11958),
  (12019, 12076),
  (12606, 12723),
  (12867, 13308),
  (13463, 13871),
  (14214, 14349),
  (14513, 14710),
  (14866, 14928),
  (14941, 15146),
  (15417, 15652),
  (17848, 18194),
  (18220, 18271),
  (18561, 18704),
  (19065, 19437),
  (20663, 20774),
  (21357, 21465),
  (21735, 21839),
  (22980, 23065),
  (23296, 23495),
  (23517, 23622),
  (23856, 24021),
  (24481, 24649),
  (24738, 24800),
  (24832, 24950),
  (25965, 26224),
  (26933, 27037),
  (27683, 27960),
  (28153, 28307),
  (28504, 28620),
  (28958, 29117),
  (29772, 29929),
  (29985, 30237),
  (30244, 30396),
  (30437, 30712),
  (30863, 30986),
  (31039, 31305),
  (31450, 31830),
  (31988, 32152),
  (321

In [39]:
# create_article_id_label_seqfor article_seq in article_final_sequence
final_submission_list =  [get_spans_from_label_seq(article_tup) for article_tup in article_final_sequence]

In [40]:
final_submission_strings = []
for fsl in final_submission_list:
    article_id = fsl[0]
    for span in fsl[1]:
        final_submission_strings.append(str(article_id) + "\t" + str(span[0]) + "\t" + str(span[1]))

In [41]:
final_submission_content = "\n".join(final_submission_strings)

In [42]:
final_submission_content = "id\tbegin_offset\tend_offset\n" + final_submission_content

In [43]:
with open("final_submission_from_flair.txt","w") as f:
    f.writelines(final_submission_content)

## Shorten the submission file

In [80]:
article_spans = open("final_submission.txt","r").readlines()

In [91]:
span_diff = 310
new_article_spans = []
for ix, article_span in enumerate(article_spans):
    if ix == 0:
        pass
    elif ix==1:
        new_article_spans.append(article_span.split("\t"))
    else:
        next_span_start_position =  int(article_span.split("\t")[1])
        prev_span_end_position = int(new_article_spans[-1][2])
        next_article_id =  int(article_span.split("\t")[0])
        prev_article_id = int(new_article_spans[-1][0])        
#         print(prev_span_end_position,next_span_start_position)
        if (next_article_id == prev_article_id):
            if (next_span_start_position - prev_span_end_position) < span_diff:
                to_insert_span = [new_article_spans[-1][0],new_article_spans[-1][1],article_span.split("\t")[2]]
                new_article_spans.pop()
                new_article_spans.append(to_insert_span)
            else:
                new_article_spans.append(article_span.split("\t"))
        else:
            new_article_spans.append(article_span.split("\t"))

In [92]:
print(len(new_article_spans))

21675


In [93]:
article_span_strings = []
for article_span in new_article_spans:
    article_span_strings.append("\t".join(article_span))

In [94]:
condensed_final_submission = "".join(article_span_strings)

In [95]:
with open("condensed_final_submission.txt","w") as f:
    f.writelines(condensed_final_submission)

## Try out sentence piece

In [127]:
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=/data/semeval-2020/task-11/datasets/all.txt --model_prefix=v16000 --character_coverage=1 --vocab_size=16000')

True

In [131]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load("v16000.model")
sp.NBestEncodeAsPieces("Federalist",5)

[['▁Federalist'],
 ['▁Federal', 'ist'],
 ['▁Federal', 'is', 't'],
 ['▁Federal', 'i', 'st'],
 ['▁', 'Federal', 'ist']]

In [130]:
for n in range(10):
    print(sp.SampleEncodeAsPieces('Federalist', -1, 0.1))

['▁', 'F', 'ed', 'e', 'ral', 'is', 't']
['▁F', 'ed', 'era', 'l', 'is', 't']
['▁Fed', 'e', 'r', 'a', 'l', 'i', 'st']
['▁F', 'e', 'der', 'al', 'ist']
['▁', 'Federal', 'is', 't']
['▁', 'Federal', 'ist']
['▁F', 'ed', 'er', 'ali', 'st']
['▁', 'F', 'e', 'der', 'al', 'is', 't']
['▁Fed', 'er', 'ali', 'st']
['▁Federalist']


In [133]:
from bpemb import BPEmb
bpemb_en = BPEmb(lang="en", dim=50)

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model


100%|██████████| 400869/400869 [00:00<00:00, 637227.84B/s]


downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d50.w2v.bin.tar.gz


100%|██████████| 1924908/1924908 [00:01<00:00, 1923637.13B/s]
/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [135]:
bpemb_en.encode("Federalist")

['▁federal', 'ist']

In [137]:
bpemb_en

BPEmb(lang=en, vs=10000, dim=50)

In [1]:
from transformers import BertTokenizer

In [2]:
bt = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
bt.tokenize("Donald")

['donald']

## Join sentence level classification with span output

In [90]:
import pandas as pd
import swifter

In [91]:
final_submission = pd.read_csv("/home/raghavan/Downloads/final_submission.txt",sep="\t")
fast_text_output = pd.read_csv("/home/raghavan/Downloads/output_dev.csv")

In [92]:
final_submission.head(5)

,id,begin_offset,end_offset
0,779309765,111,133
1,779309765,142,150
2,779309765,409,454
3,779309765,615,692
4,779309765,1514,1526


In [93]:
fast_text_output.head(5)

,article_id,sentence_id,text,start,end,label
0,article788271400.txt,0,"Are You Kidding Me, Ted Cruz?",1,30,False
1,article788271400.txt,1,".Don't ""Blame The Police Officer"" Who Admitted...",31,98,False
2,article788271400.txt,2,"..Dallas, TX — In a repugnant and outright dis...",99,314,False
3,article788271400.txt,3,".During an interview over the weekend, Cruz sa...",315,472,False
4,article788271400.txt,4,.FOX 26 asked Cruz to respond to his Democrati...,473,593,False


In [94]:
final_submission["article_id"] = final_submission["id"]
fast_text_output["article_file_name"] = fast_text_output["article_id"]
fast_text_output["article_id"] = fast_text_output["article_file_name"].swifter.apply(lambda x: int(x.replace("article","").replace(".txt","")))

/home/raghavan/anaconda3/envs/semeval_2020/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [95]:
fast_text_output.dtypes

article_id            int64
sentence_id           int64
text                 object
start                 int64
end                   int64
label                  bool
article_file_name    object
dtype: object

In [96]:
final_submission.dtypes

id              int64
begin_offset    int64
end_offset      int64
article_id      int64
dtype: object

In [97]:
fast_text_output_only_prop = fast_text_output[fast_text_output["label"] == True]

In [98]:
fasttext_span_joined = fast_text_output_only_prop.merge(final_submission,on="article_id",how="outer")

In [111]:
def is_span_with_in(row):
#     print(row)
    if (row.begin_offset >= row.start - 20) and (row.end_offset -20 <= row.end):
        return True
    else:
        return False

In [112]:
# fasttext_span_joined["is_span_within"] = 
fasttext_span_joined["is_span_with"] = fasttext_span_joined.swifter.apply(is_span_with_in,axis=1)

In [113]:
to_submit_row =  fasttext_span_joined[fasttext_span_joined["is_span_with"] == True]

In [114]:
to_submit_row

,article_id,sentence_id,text,start,end,label,article_file_name,id,begin_offset,end_offset,is_span_with
2282,788271400,32.0,".In Cruz’s eyes, Guyger going into an apartmen...",3314.0,3530.0,True,article788271400.txt,788271400,3445,3451,True
3418,738447109,33.0,".While many are happy that Tillerson is gone, ...",3517.0,3790.0,True,article738447109.txt,738447109,3595,3632,True
4553,778094905,31.0,.Political correctness needs to be thrown out ...,3544.0,3726.0,True,article778094905.txt,778094905,3595,3632,True
6818,832908905,18.0,.“Attorney General Barr must not give Presiden...,2382.0,2662.0,True,article832908905.txt,832908905,2500,2510,True
7954,832908905,19.0,.The Democratic leaders said Mueller’s investi...,2663.0,2917.0,True,article832908905.txt,832908905,2878,2922,True
...,...,...,...,...,...,...,...,...,...,...,...
301922,999000851,25.0,".This means that Acosta, or any other reporter...",3260.0,3467.0,True,article999000851.txt,999000851,3445,3451,True
306459,999000878,22.0,".Not only that, he used his hand to forcefully...",2825.0,2918.0,True,article999000878.txt,999000878,2878,2922,True
310994,763280007,17.0,".Meanwhile, Britain has a steadily lengthening...",1497.0,1611.0,True,article763280007.txt,763280007,1514,1526,True
310995,763280007,17.0,".Meanwhile, Britain has a steadily lengthening...",1497.0,1611.0,True,article763280007.txt,763280007,1543,1551,True


In [115]:
to_submit_row.count()

article_id           167
sentence_id          167
text                 167
start                167
end                  167
label                167
article_file_name    167
id                   167
begin_offset         167
end_offset           167
is_span_with         167
dtype: int64